In [234]:
## making the linear regression to predict the future value of cars
## making imports
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import sqlite3

In [235]:
#loading file ~~ can be adjusted later but using columns from sql database:
#price, year, manufacturer, condition, cylinders, odometer, title_status, transmission, size, state, posting_date
file_path = Path("../data/vehicles_clean.csv")


In [236]:
#making dataframe
used_cars_df = pd.read_csv(file_path)

used_cars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80170 entries, 0 to 80169
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         80170 non-null  int64  
 1   year          80170 non-null  int64  
 2   manufacturer  80170 non-null  object 
 3   condition     80170 non-null  object 
 4   cylinders     80170 non-null  object 
 5   fuel          80170 non-null  object 
 6   odometer      80170 non-null  float64
 7   title_status  80170 non-null  object 
 8   transmission  80170 non-null  object 
 9   drive         80170 non-null  object 
 10  size          80170 non-null  object 
 11  type          80170 non-null  object 
 12  paint_color   80170 non-null  object 
 13  state         80170 non-null  object 
 14  posting_date  80170 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 9.2+ MB


In [237]:
#dropping cars that are free
mask = used_cars_df['price'] == 0
used_cars_df = used_cars_df.loc[~mask]

In [238]:
#dropping un-drivable cars
# mask2 = used_cars_df['title_status'] == 'parts only','salvage'
# used_cars_df = used_cars_df.loc[~mask2]
used_cars_df.condition.unique()

array(['excellent', 'good', 'new', 'fair', 'like new', 'salvage'],
      dtype=object)

In [239]:
used_cars_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76692 entries, 0 to 80168
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         76692 non-null  int64  
 1   year          76692 non-null  int64  
 2   manufacturer  76692 non-null  object 
 3   condition     76692 non-null  object 
 4   cylinders     76692 non-null  object 
 5   fuel          76692 non-null  object 
 6   odometer      76692 non-null  float64
 7   title_status  76692 non-null  object 
 8   transmission  76692 non-null  object 
 9   drive         76692 non-null  object 
 10  size          76692 non-null  object 
 11  type          76692 non-null  object 
 12  paint_color   76692 non-null  object 
 13  state         76692 non-null  object 
 14  posting_date  76692 non-null  object 
dtypes: float64(1), int64(2), object(12)
memory usage: 9.4+ MB


In [240]:
# Cleaning the Data
used_cars_df = used_cars_df.drop('posting_date', axis=1)

In [241]:
# Label encoding manufacturer, cylinders, fuel, transmission, drive, size, type, paint_color, state
used_cars_df['manufacturer'] = used_cars_df['manufacturer'].astype('category')
used_cars_df['cylinders'] = used_cars_df['cylinders'].astype('category')
used_cars_df['fuel'] = used_cars_df['fuel'].astype('category')
used_cars_df['transmission'] = used_cars_df['transmission'].astype('category')
used_cars_df['drive'] = used_cars_df['drive'].astype('category')
used_cars_df['size'] = used_cars_df['size'].astype('category')
used_cars_df['type'] = used_cars_df['type'].astype('category')
used_cars_df['paint_color'] = used_cars_df['paint_color'].astype('category')
used_cars_df['state'] = used_cars_df['state'].astype('category')
used_cars_df['year'] = used_cars_df['year'].astype('category')

In [242]:
#label encoding continued
used_cars_df["manufacturer"] = used_cars_df["manufacturer"].cat.codes
used_cars_df["cylinders"] = used_cars_df["cylinders"].cat.codes
used_cars_df["fuel"] = used_cars_df["fuel"].cat.codes
used_cars_df["transmission"] = used_cars_df["transmission"].cat.codes
used_cars_df["drive"] = used_cars_df["drive"].cat.codes
used_cars_df["size"] = used_cars_df["size"].cat.codes
used_cars_df["type"] = used_cars_df["type"].cat.codes
used_cars_df["paint_color"] = used_cars_df["paint_color"].cat.codes
used_cars_df["state"] = used_cars_df["state"].cat.codes
used_cars_df["year"] = used_cars_df["year"].cat.codes

In [243]:
used_cars_df.head(5)

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,15000,94,13,excellent,5,2,128000.0,clean,0,2,1,10,0,1
1,19900,85,13,good,6,0,88000.0,clean,0,0,1,8,1,1
2,14000,93,16,excellent,5,2,95000.0,clean,0,1,1,5,9,1
3,22500,82,13,good,6,0,144700.0,clean,1,2,1,10,10,1
4,15000,98,10,excellent,6,2,90000.0,rebuilt,0,2,2,9,5,1


In [244]:
# Importing Encoder
or_enc = OrdinalEncoder()
ohe = OneHotEncoder()

In [245]:
# Ordinal Encoding
used_cars_df['title_status'] = or_enc.fit_transform(used_cars_df[['title_status']])
used_cars_df['condition'] = or_enc.fit_transform(used_cars_df[['condition']])


In [246]:
used_cars_df.head(20)

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color,state
0,15000,94,13,0.0,5,2,128000.0,0.0,0,2,1,10,0,1
1,19900,85,13,2.0,6,0,88000.0,0.0,0,0,1,8,1,1
2,14000,93,16,0.0,5,2,95000.0,0.0,0,1,1,5,9,1
3,22500,82,13,2.0,6,0,144700.0,0.0,1,2,1,10,10,1
4,15000,98,10,0.0,6,2,90000.0,4.0,0,2,2,9,5,1
5,3000,85,8,2.0,5,2,176144.0,0.0,0,1,2,5,9,1
6,9500,84,8,0.0,5,2,30376.0,0.0,0,1,2,5,1,1
7,17500,89,38,2.0,5,2,201300.0,0.0,1,0,1,6,0,1
11,4000,87,20,2.0,5,2,281000.0,0.0,0,2,2,0,0,1
12,22950,95,13,0.0,6,2,166380.0,0.0,0,0,1,10,0,1


In [247]:
#train & test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, random_state = 42)

print("training shape: ", X_train.shape, y_train.shape)
print("testing shape: ", X_test.shape, y_test.shape)

training shape:  (61353, 13) (61353,)
testing shape:  (15339, 13) (15339,)


In [248]:
# the model
lr = LinearRegression()

In [249]:
#fit the model
lr.fit(X_train,y_train)

LinearRegression()

In [250]:
#predict on testing set
y_pred = lr.predict(X_test)

In [251]:
# Evaluate model performance
r2 = r2_score(y_test,y_pred)
mse = mean_squared_error(y_test,y_pred)
mae = mean_absolute_error(y_test,y_pred)
intercept = lr.intercept_

print("r2 score: ", r2)
print("mean squared error: ", mse)
print("mean absolute error: ", mae)
print("intercept: ", intercept)

r2 score:  -9.857299002780096e-05
mean squared error:  991892137149092.2
mean absolute error:  339056.5780333381
intercept:  -16128.18659217394
